# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `recipes_sample.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [2]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [72]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /home/fampkin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/fampkin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [73]:
words = list()

for description in recipes['description']:
    if isinstance(description, str):
        tokens = nltk.word_tokenize(description)
        words.extend(tokens)

ds = pd.Series(words)
ds.head()


0          an
1    original
2      recipe
3     created
4          by
dtype: object

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [39]:
from nltk.metrics import edit_distance

random_words = ds.sample(10).values

for i in range(0, 10, 2):
    word1 = random_words[i]
    word2 = random_words[i+1]
    distance = edit_distance(word1, word2)
    print(f"\nПара {i//2 + 1}:")
    print(f"Слово 1: '{word1}'")
    print(f"Слово 2: '{word2}'")
    print(f"Расстояние редактирования: {distance}")


Пара 1:
Слово 1: ''m'
Слово 2: 'bulk'
Расстояние редактирования: 4

Пара 2:
Слово 1: 'and'
Слово 2: 'href='
Расстояние редактирования: 5

Пара 3:
Слово 1: 'isle'
Слово 2: 'title'
Расстояние редактирования: 2

Пара 4:
Слово 1: 'turning'
Слово 2: 'i'
Расстояние редактирования: 6

Пара 5:
Слово 1: 'fresh'
Слово 2: 'cookbook'
Расстояние редактирования: 8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [41]:
k = 10

word = 'cooking'

def get_k_nearest_words(word, words, k):
    words_dist = dict()
    for w in words:
        distance = edit_distance(word, w)
        words_dist[w] = distance
    words_dist = sorted(words_dist.items(), key=lambda x: x[1])
    return [d[0] for d in words_dist[:k]]

nearest_words = get_k_nearest_words(word, words, k)
print(f"Ближайшие слова к '{word}': {nearest_words}")



Ближайшие слова к 'cooking': ['cooking', 'looking', 'cookin', 'cooling', 'ccoking', "'cooking", 'cookings', 'cooking_', 'choking', 'cookiing']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [47]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer

nltk.download('wordnet')

stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/fampkin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
df = pd.DataFrame({'word': words})
df['stemmed_word'] = df['word'].apply(lambda x: stemmer.stem(x))
df['normalized_word'] = df['word'].apply(lambda x: lemmatizer.lemmatize(x))

df.set_index('word', inplace=True)
df.head()



,stemmed_word,normalized_word
word,,
an,an,an
original,origin,original
recipe,recip,recipe
created,creat,created
by,by,by


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [55]:
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fampkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
filtered_words = ds[~ds.isin(stop_words)]
filtered_words.head()

print("\nСтатистика:")
print(f"Всего слов: {len(ds)}")
print(f"Слов после удаления стоп-слов: {len(filtered_words)}")
print(f"Доля стоп-слов: {((len(ds) - len(filtered_words)) / len(ds)) * 100:.2f}%")

print("\nПримеры слов после фильтрации:")
print(filtered_words.head())



Статистика:
Всего слов: 1242181
Слов после удаления стоп-слов: 742017
Доля стоп-слов: 40.26%

Примеры слов после фильтрации:
1    original
2      recipe
3     created
5        chef
6       scott
dtype: object


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()



In [65]:
random_recipes = recipes.sample(5)

vectors = tfidf.fit_transform(random_recipes['description'].fillna(''))

dense_vectors = vectors.todense()

vector_df = pd.DataFrame(
    dense_vectors,
    index=random_recipes['name'],
    columns=tfidf.get_feature_names_out()
)

vector_df.head()

,adopt,an,and,another,are,at,banana,bits,bloomingdale,bread,...,up,use,very,want,was,way,we,while,winter,with
name,,,,,,,,,,,,,,,,,,,,,
spicy szechuan noodles dan dan mian,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sweet and salty cereal bars,0.000000,0.000000,0.113606,0.169634,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.169634,0.169634,0.000000,0.000000,0.000000,0.339267,0.000000,0.169634,0.000000,0.000000
granny s tater soup,0.153034,0.000000,0.000000,0.000000,0.000000,0.153034,0.000000,0.000000,0.153034,0.000000,...,0.000000,0.000000,0.123467,0.153034,0.153034,0.000000,0.306068,0.000000,0.153034,0.000000
low fat morning glory muffins,0.000000,0.000000,0.371921,0.000000,0.138836,0.000000,0.000000,0.138836,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.555345
aunt muriel s banana bread,0.000000,0.249227,0.166910,0.000000,0.000000,0.000000,0.249227,0.000000,0.000000,0.249227,...,0.000000,0.000000,0.201075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [69]:
from scipy.spatial.distance import cosine
import numpy as np

n_recipes = len(random_recipes)
distances = np.zeros((n_recipes, n_recipes))

for i in range(n_recipes):
    for j in range(n_recipes):
        distances[i, j] = cosine(vector_df.iloc[i], vector_df.iloc[j])

distance_df = pd.DataFrame(
    distances,
    index=random_recipes['name'],
    columns=random_recipes['name']
)

distance_df.head()

name,spicy szechuan noodles dan dan mian,sweet and salty cereal bars,granny s tater soup,low fat morning glory muffins,aunt muriel s banana bread
name,,,,,
spicy szechuan noodles dan dan mian,0.0,1.000000,1.000000,1.000000,1.000000
sweet and salty cereal bars,1.0,0.000000,0.885836,0.875302,0.981038
granny s tater soup,1.0,0.885836,0.000000,0.980941,0.975174
low fat morning glory muffins,1.0,0.875302,0.980941,0.000000,0.937923
aunt muriel s banana bread,1.0,0.981038,0.975174,0.937923,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [71]:
distances_no_diagonal = distance_df.copy()
np.fill_diagonal(distances_no_diagonal.values, 1.0)

min_distance = distances_no_diagonal.min().min()
most_similar_pair = np.where(distances_no_diagonal == min_distance)
recipe1 = distances_no_diagonal.index[most_similar_pair[0][0]]
recipe2 = distances_no_diagonal.columns[most_similar_pair[1][0]]

print("Анализ схожести рецептов:")
print(f"\nНаиболее похожая пара рецептов:")
print(f"1. {recipe1}")
print(f"2. {recipe2}")
print(f"Косинусное расстояние между ними: {min_distance:.3f}")

print("\nОписания этих рецептов:")
print(f"\nРецепт 1: {recipes[recipes['name'] == recipe1]['description'].values[0]}")
print(f"\nРецепт 2: {recipes[recipes['name'] == recipe2]['description'].values[0]}")

print("\nОбъяснение схожести:")
common_words = vector_df.loc[[recipe1, recipe2]].multiply(vector_df.loc[[recipe1, recipe2]]).sum()
top_common_words = common_words[common_words > 0].sort_values(ascending=False).head(5)
print("\nОбщие важные слова в этих рецептах:")
print(top_common_words)

Анализ схожести рецептов:

Наиболее похожая пара рецептов:
1. sweet and salty cereal bars
2. low fat morning glory muffins
Косинусное расстояние между ними: 0.875

Описания этих рецептов:

Рецепт 1: tasty way to satisfy those "sweet and salty" cravings. :) i found this recipe online while searching for a way to use up the rest of the crispix cereal i had purchased for another recipe.

Рецепт 2: lovely muffins packed with succulent bits of raisins and carrot, studded with crunchy sunflower seeds, and permeated with the flavors of orange and cinnamon. served with fresh fruit and scrambled eggs, these are great for a special breakfast!

Объяснение схожести:

Общие важные слова в этих рецептах:
with      0.308408
and       0.151231
recipe    0.115102
way       0.115102
for       0.087469
dtype: float64




1. **Степень схожести:**
   - Косинусное расстояние 0.875 указывает на относительно низкую схожесть рецептов (чем ближе к 1, тем менее похожи рецепты)
   - Это объяснимо, так как рецепты описывают разные блюда: батончики из хлопьев и маффины

2. **Анализ общих слов:**
   - Наиболее значимые общие слова ("with", "and", "recipe", "way", "for") являются скорее структурными, чем содержательными
   - Это указывает на схожесть в стиле описания, но не в самих рецептах

3. **Различия в рецептах:**
   - Первый рецепт о сладко-соленых батончиках из хлопьев
   - Второй рецепт о низкокалорийных маффинах с изюмом, морковью и семечками
   - Ингредиенты и способы приготовления существенно различаются

4. **Почему все же есть некоторая схожесть:**
   - Оба рецепта относятся к категории выпечки/снеков
   - В обоих описаниях используется позитивная лексика ("tasty", "lovely")
   - Оба рецепта включают описание текстур и вкусовых качеств
   - Оба являются закусками/десертами

Таким образом, несмотря на то что рецепты описывают разные блюда, они имеют схожую структуру описания и относятся к похожей категории продуктов, что объясняет наличие общих слов в их векторных представлениях.
